In [0]:
# Try out some data augmentation to make it better.
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout
from keras.optimizers import Adam
from keras.models import load_model
from sklearn.metrics import accuracy_score

Functions to clean up the work flow

In [0]:
# Loading in data
def load_data(file_path="FashionMNIST/data/fashion_mnist_scaled.npz"):
    data = np.load(file_path, allow_pickle=True)
    X_train, y_train, X_test, y_test = [data[file] for file in data.files]
    return X_train, y_train, X_test, y_test


# Getting an augmented dataset
def augmenting_data(gen, X, y, num_augmented_imgs=5):
    aug_iter = gen.flow(X, y, batch_size=len(X))
    data = [next(aug_iter) for num in range(num_augmented_imgs)]
    y1 = [data[num][1] for num in range(num_augmented_imgs)]
    x = [data[num][0] for num in range(num_augmented_imgs)]
    return np.concatenate([np.vstack(x), X]), np.concatenate([np.vstack(y1).reshape(-1), y])


In [0]:
# Loading in data
X_train, y_train, X_test, y_test = load_data(file_path="/content/fashion_mnist_scaled.npz")

In [0]:
# Creating image generator
gen = ImageDataGenerator(rotation_range=10, width_shift_range=.1, zoom_range=.1,
                         horizontal_flip=True)

In [0]:
# Creating augmented dataset
X_train_augmented, y_train_augmented = augmenting_data(gen, X_train, y_train, 2)

In [0]:
# Callbacks for training
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_delta=1e-4, mode='min')
callbacks = [es, mcp_save, reduce_lr_loss]

In [0]:
# Making a deeper model architecture
model = Sequential([
    Conv2D(120, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(),

    Conv2D(60, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(),

    Conv2D(30, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(10, activation='softmax')
])

In [0]:
# Compiling the simple model
model.compile(Adam(.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Fitting model with specifying validation set
model.fit(X_train_augmented, y_train_augmented, validation_split=.1, epochs=100,
           callbacks=callbacks, verbose=2)

Train on 162000 samples, validate on 18000 samples
Epoch 1/100
 - 26s - loss: 0.4784 - accuracy: 0.8256 - val_loss: 0.3086 - val_accuracy: 0.8871
Epoch 2/100
 - 20s - loss: 0.3277 - accuracy: 0.8800 - val_loss: 0.2652 - val_accuracy: 0.9064
Epoch 3/100
 - 20s - loss: 0.2877 - accuracy: 0.8943 - val_loss: 0.2475 - val_accuracy: 0.9123
Epoch 4/100
 - 20s - loss: 0.2643 - accuracy: 0.9027 - val_loss: 0.2256 - val_accuracy: 0.9176
Epoch 5/100
 - 20s - loss: 0.2471 - accuracy: 0.9082 - val_loss: 0.2227 - val_accuracy: 0.9203
Epoch 6/100
 - 20s - loss: 0.2332 - accuracy: 0.9133 - val_loss: 0.2345 - val_accuracy: 0.9165
Epoch 7/100
 - 20s - loss: 0.2208 - accuracy: 0.9180 - val_loss: 0.2234 - val_accuracy: 0.9214
Epoch 8/100
 - 20s - loss: 0.2110 - accuracy: 0.9218 - val_loss: 0.2380 - val_accuracy: 0.9188
Epoch 9/100
 - 20s - loss: 0.2032 - accuracy: 0.9246 - val_loss: 0.2345 - val_accuracy: 0.9150
Epoch 10/100
 - 20s - loss: 0.1942 - accuracy: 0.9279 - val_loss: 0.2260 - val_accuracy: 0.923

In [0]:
# Lets go even deeper...
model2 = Sequential([
    Conv2D(240, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(.3),

    Conv2D(120, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(),
    Dropout(.3),

    Conv2D(60, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(),

    Conv2D(30, (3, 3), padding='same', activation='relu'),
    Flatten(),
    Dense(10, activation='softmax')
])

In [0]:
# Compiling the model
model2.compile(Adam(.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [30]:
# Fitting model with specifying validation set
model2.fit(X_train_augmented, y_train_augmented, validation_split=.1, epochs=100,
           callbacks=callbacks, verbose=2)
# Previous was 94.07 something

Train on 162000 samples, validate on 18000 samples
Epoch 1/100
 - 30s - loss: 0.4815 - accuracy: 0.8209 - val_loss: 0.3128 - val_accuracy: 0.8858
Epoch 2/100
 - 30s - loss: 0.3344 - accuracy: 0.8747 - val_loss: 0.2581 - val_accuracy: 0.9059
Epoch 3/100
 - 29s - loss: 0.3013 - accuracy: 0.8871 - val_loss: 0.2426 - val_accuracy: 0.9081
Epoch 4/100
 - 29s - loss: 0.2807 - accuracy: 0.8944 - val_loss: 0.2183 - val_accuracy: 0.9195
Epoch 5/100
 - 29s - loss: 0.2683 - accuracy: 0.8986 - val_loss: 0.2133 - val_accuracy: 0.9218
Epoch 6/100
 - 29s - loss: 0.2598 - accuracy: 0.9015 - val_loss: 0.2069 - val_accuracy: 0.9248
Epoch 7/100
 - 29s - loss: 0.2524 - accuracy: 0.9049 - val_loss: 0.2096 - val_accuracy: 0.9230
Epoch 8/100
 - 29s - loss: 0.2470 - accuracy: 0.9061 - val_loss: 0.2166 - val_accuracy: 0.9222
Epoch 9/100
 - 29s - loss: 0.2434 - accuracy: 0.9087 - val_loss: 0.1968 - val_accuracy: 0.9284
Epoch 10/100
 - 29s - loss: 0.2393 - accuracy: 0.9095 - val_loss: 0.2048 - val_accuracy: 0.925

In [0]:
# Lets go even deeper... and deeper...
model3 = Sequential([
    Conv2D(480, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(),
    Dropout(.3),

    Conv2D(240, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(.3),

    Conv2D(120, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(.3),

    Conv2D(60, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(),

    Conv2D(30, (3, 3), padding='same', activation='relu'),
    Flatten(),
    Dense(10, activation='softmax')
])

In [0]:
# Compiling the model
model3.compile(Adam(.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [40]:
# Fitting model with specifying validation set
model3.fit(X_train_augmented, y_train_augmented, validation_split=.1, epochs=100,
           callbacks=callbacks, verbose=2)

Train on 162000 samples, validate on 18000 samples
Epoch 1/100
 - 51s - loss: 0.1153 - accuracy: 0.9568 - val_loss: 0.1391 - val_accuracy: 0.9517
Epoch 2/100
 - 50s - loss: 0.1129 - accuracy: 0.9574 - val_loss: 0.1389 - val_accuracy: 0.9526
Epoch 3/100
 - 51s - loss: 0.1138 - accuracy: 0.9577 - val_loss: 0.1398 - val_accuracy: 0.9521
Epoch 4/100
 - 51s - loss: 0.1128 - accuracy: 0.9574 - val_loss: 0.1394 - val_accuracy: 0.9519
Epoch 5/100
 - 51s - loss: 0.1156 - accuracy: 0.9567 - val_loss: 0.1392 - val_accuracy: 0.9519
Epoch 6/100
 - 51s - loss: 0.1138 - accuracy: 0.9572 - val_loss: 0.1390 - val_accuracy: 0.9525
Epoch 7/100
 - 51s - loss: 0.1121 - accuracy: 0.9580 - val_loss: 0.1414 - val_accuracy: 0.9515
Epoch 8/100
 - 51s - loss: 0.1127 - accuracy: 0.9578 - val_loss: 0.1396 - val_accuracy: 0.9516
Epoch 9/100
 - 51s - loss: 0.1133 - accuracy: 0.9573 - val_loss: 0.1401 - val_accuracy: 0.9524
Epoch 10/100
 - 51s - loss: 0.1119 - accuracy: 0.9585 - val_loss: 0.1376 - val_accuracy: 0.952

In [0]:
# Saving out the best model
model3.save("/content/deep_cnn_best.h5")

In [0]:
# Using previous models architecture to train on initial data. Not the augmented
# data. My thoughts are that it wont quite be as good. Probably overfit.
prev_mod = load_model("/content/deep_cnn_best.h5")

In [0]:
# Creating new model object with previous architecture
reg_data_mod = Sequential.from_config(prev_mod.get_config())

In [50]:
# Lets see if the architecture matches up
reg_data_mod.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 28, 28, 480)       4800      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 14, 14, 480)       0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 14, 14, 480)       0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 14, 14, 240)       1037040   
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 7, 7, 240)         0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 7, 7, 240)         0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 7, 7, 120)        

In [0]:
# Looks good. Lets go ahead and train it
reg_data_mod.compile(Adam(0.001), loss="sparse_categorical_crossentropy", 
                     metrics=["accuracy"])

In [54]:
reg_data_mod.fit(X_train, y_train, epochs=100, callbacks=callbacks, verbose=2,
                   validation_split=.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/100
 - 17s - loss: 0.5365 - accuracy: 0.8011 - val_loss: 0.3768 - val_accuracy: 0.8617
Epoch 2/100
 - 17s - loss: 0.3425 - accuracy: 0.8744 - val_loss: 0.2876 - val_accuracy: 0.8943
Epoch 3/100
 - 17s - loss: 0.2950 - accuracy: 0.8917 - val_loss: 0.2622 - val_accuracy: 0.9017
Epoch 4/100
 - 17s - loss: 0.2748 - accuracy: 0.8987 - val_loss: 0.2497 - val_accuracy: 0.9093
Epoch 5/100
 - 17s - loss: 0.2532 - accuracy: 0.9056 - val_loss: 0.2485 - val_accuracy: 0.9065
Epoch 6/100
 - 17s - loss: 0.2405 - accuracy: 0.9116 - val_loss: 0.2245 - val_accuracy: 0.9182
Epoch 7/100
 - 17s - loss: 0.2271 - accuracy: 0.9154 - val_loss: 0.2265 - val_accuracy: 0.9167
Epoch 8/100
 - 17s - loss: 0.2165 - accuracy: 0.9211 - val_loss: 0.2247 - val_accuracy: 0.9157
Epoch 9/100
 - 17s - loss: 0.2105 - accuracy: 0.9228 - val_loss: 0.2220 - val_accuracy: 0.9173
Epoch 10/100
 - 17s - loss: 0.2038 - accuracy: 0.9246 - val_loss: 0.2144 - val_accuracy: 0.9240


In [0]:
# Using best model to predict on the test set
predictions = model3.predict_classes(X_test)

In [59]:
accuracy_score(y_test, predictions)

0.9396